<a href="https://colab.research.google.com/github/zerotodeeplearning/ztdl-masterclasses/blob/master/solutions_do_not_open/Pre-trained_Models_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Copyright 2020 Catalit LLC.

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Pre-trained Models

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# sports_images_path = tf.keras.utils.get_file(
#     'sports_images',
#     'https://archive.org/download/ztdl_sports_images/sports_images.tgz',
#      untar=True)

In [ ]:
![[ ! -f sports_images.tar.gz ]] && gsutil cp gs://ztdl-datasets/sports_images.tar.gz .
![[ ! -d sports_images ]] && echo "Extracting images..." && tar zxf sports_images.tar.gz
sports_images_path  = './sports_images'

In [ ]:
train_path = os.path.join(sports_images_path, 'train')
test_path = os.path.join(sports_images_path, 'test')

In [ ]:
batch_size = 16
img_size = 224

In [ ]:
train_datagen = ImageDataGenerator() \
  .flow_from_directory(train_path, 
                       target_size = (img_size, img_size),
                       batch_size = batch_size,
                       class_mode = 'sparse')

In [ ]:
try:
  assert(train_datagen.samples == 11414)
except:
  raise Exception("Found less images than expected. Please remove the files and download again.")

In [ ]:
classes_dict = train_datagen.class_indices
classes = list(classes_dict.keys())
classes

In [ ]:
batch, labels = train_datagen.next()

In [ ]:
batch.shape

In [ ]:
labels.shape

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(len(batch)):
  plt.subplot(4, 4, i+1)
  plt.imshow(batch[i].astype('int'))
  plt.title(classes[int(labels[i])])
  plt.axis('off')

plt.tight_layout()

### Pre-trained model

Let's use a Resnet50 model to classify the images without any training.

In [ ]:
from PIL import Image
from io import BytesIO
from IPython.display import HTML
import base64

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_input_resnet50
from tensorflow.keras.applications.resnet50 import decode_predictions as decode_predictions_resnet50

In [ ]:
model = ResNet50(weights='imagenet')

In [ ]:
batch_preprocessed = preprocess_input_resnet50(batch.copy())

In [ ]:
predictions = model.predict(batch_preprocessed)

In [ ]:
decoded_top_3 = decode_predictions_resnet50(predictions, top=3)

In [ ]:
def image_formatter(a):
  im = Image.fromarray(a)
  im.thumbnail((28, 28), Image.LANCZOS)
  with BytesIO() as buffer:
    im.save(buffer, 'jpeg')
    im_base64 = base64.b64encode(buffer.getvalue()).decode()
  return f'<img src="data:image/jpeg;base64,{im_base64}">'

In [ ]:
def display_batch(batch, decoded_top_3):
  res = []
  for i, top3 in enumerate(decoded_top_3):
    im = image_formatter(batch[i].astype('uint8'))
    cl = classes[int(labels[i])]
    line = [im, cl]
    for item in top3:
      line = line + list(item[1:])
    res.append(line)


  res_df = pd.DataFrame(res,
                        columns=['image', 'ground_truth',
                                'top_1', 'prob_1',
                                'top_2', 'prob_2',
                                'top_3', 'prob_3'])

  
  return res_df.style.bar(color='lightgreen', vmin=0, vmax=1)

In [ ]:
display_batch(batch, decoded_top_3)

### Exercise 1:

Use a different pre-trained model from the ones provided at: https://keras.io/applications/

Do the predictions match?

You will need to:
- import the pre-trained model
- import the corresponding `preprocess_input` and `decode_predictions`
- check the correct imput shape for your chosen model and possibly re-load a new batch with updated image size
- pipe the batch through the predict function and display the results

In [ ]:
from tensorflow.keras.applications.nasnet import NASNetMobile
from tensorflow.keras.applications.nasnet import preprocess_input as preprocess_input_nasnet
from tensorflow.keras.applications.nasnet import decode_predictions as decode_predictions_nasnet

In [ ]:
model = NASNetMobile(weights='imagenet')

In [ ]:
batch_preprocessed = preprocess_input_nasnet(batch.copy())

In [ ]:
predictions = model.predict(batch_preprocessed)

In [ ]:
decoded_top_3 = decode_predictions_nasnet(predictions, top=3)

In [ ]:
display_batch(batch, decoded_top_3)